# 1. Parameters

In [1]:
# Defaults
cases_dir = 'cases/unset'
reference_file = 'references/NC_045512.gbk.gz'
input_files_all = 'input/input-files.tsv'
iterations = 3
mincov = 10
ncores = 32
number_samples = 10
build_tree = False

In [2]:
# Parameters
cases_dir = "cases/case-50000"
iterations = 3
number_samples = 50000
build_tree = False


In [3]:
from pathlib import Path
from shutil import rmtree
from os import makedirs
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

if cases_dir_path.exists():
    rmtree(cases_dir_path)
    
if not cases_dir_path.exists():
    makedirs(cases_dir_path)

input_files_all = Path(input_files_all)
reference_file = Path(reference_file)

case_name = str(cases_dir_path.name)
reference_name = reference_file.name.split('.')[0]

cases_input = cases_dir_path / 'input-files-case.tsv'
index_path = cases_dir_path / 'index'
benchmark_path = cases_dir_path / 'index-info.tsv'
output_tree = cases_dir_path / 'tree.tre'

# 2. Create subset input

In [4]:
import pandas as pd

all_input_df = pd.read_csv(input_files_all, sep='\t')
all_input_total = len(all_input_df)

subset_input_df = all_input_df.head(number_samples)
subset_input_total = len(subset_input_df)

subset_input_df.to_csv(cases_input, sep='\t', index=False)

print(f'Wrote {subset_input_total}/{all_input_total} samples to {cases_input}')

Wrote 50000/100000 samples to cases/case-50000/input-files-case.tsv


# 2. Index genomes

In [5]:
!gdi --version

gdi, version 0.3.0.dev12


## 2.1. Index reads

In [6]:
results_handler = gdi_benchmark.BenchmarkResultsHandler(name=case_name)
benchmarker = gdi_benchmark.IndexBenchmarker(benchmark_results_handler=results_handler,
                                             index_path=index_path, input_files_file=cases_input,
                                             reference_file=reference_file, mincov=mincov,
                                             build_tree=build_tree,
                                             ncores=ncores)

benchmark_df = benchmarker.benchmark(iterations=iterations)


Iteration 1 of index/analysis of 50000 samples with 32 cores
Removing any extra snakemake directories: ['snakemake-assemblies.1631178584.174109']


Creating new index: [gdi init cases/case-50000/index]


Creating a new index took 2.80 seconds
Analysis running: [gdi --project-dir cases/case-50000/index --ncores 32 analysis --use-conda --no-load-data --reference-file references/NC_045512.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/case-50000/input-files-case.tsv]


Process Process-6:


Process Process-5:


...

  File "/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-sars-cov-2/lib/python3.8/site-packages/psutil/__init__.py", line 763, in io_counters
    return self._proc.io_counters()


  File "/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-sars-cov-2/lib/python3.8/site-packages/psutil/_pslinux.py", line 1576, in wrapper
    return fun(self, *args, **kwargs)


  File "/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-sars-cov-2/lib/python3.8/site-packages/psutil/_pslinux.py", line 1741, in io_counters
    for line in f:


KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
benchmark_df

In [ ]:
benchmark_df.to_csv(benchmark_path, sep='\t', index=False)

# 3. Export trees

In [ ]:
if build_tree:
    !gdi --project-dir {index_path} export tree {reference_name} > {output_tree}
    print(f'Wrote tree to {output_tree}')
else:
    print(f'build_tree={build_tree} so no tree to export')